In [241]:
import csv,sys
import math,random
import subprocess
import numpy as np
import pandas as pd
import cvxopt
import cvxopt.solvers
from sklearn.metrics import roc_curve
from sklearn.svm import OneClassSVM

In [242]:
data = pd.read_csv("DSL-StrongPasswordData.csv")

In [243]:
subjects = data["subject"].unique()

In [244]:
def evaluateEER(user_scores, imposter_scores):
    #print user_scores
    labels = [0]*len(user_scores) + [1]*len(imposter_scores)
    fpr, tpr, thresholds = roc_curve(labels, user_scores + imposter_scores)
    missrates = 1 - tpr
    farates = fpr
    dists = missrates - farates
    idx1 = np.argmin(dists[dists >= 0])
    idx2 = np.argmax(dists[dists < 0])
    x = [missrates[idx1], farates[idx1]]
    y = [missrates[idx2], farates[idx2]]
    a = ( x[0] - x[1] ) / ( y[1] - x[1] - y[0] + x[0] )
    eer = x[0] + a * ( y[0] - x[0] )
    return eer

In [245]:
def evaluate():
    eers = []
    for subject in subjects:
            
        user_scores = []
        imposter_scores = []
    
        # Consider current subject as genuine and rest as imposters
        genuine_user_data = data.loc[data.subject == subject, "H.period":"H.Return"]
        imposter_data = data.loc[data.subject != subject, :]
    
        # genuine user's first 200 time vectors for training
        train = genuine_user_data[:200]
        train = train.values
        train = np.array(train)
        #labels = np.ones(len(train),dtype = np.float64)
        #print type(labels[0])
        test_genuine = genuine_user_data[200:]
        test_genuine = test_genuine.values
        test_genuine = np.array(test_genuine)
        test_imposter = imposter_data.groupby("subject").head(5).loc[:, "H.period":"H.Return"]
        test_imposter = test_imposter.values
        test_imposter = np.array(test_imposter)
        clf = OneClassSVM(kernel='rbf',gamma=0.1)
        clf.fit(train)
        #print len(test_genuine)
        #print len(test_imposter)
        user_scores = clf.decision_function(test_genuine)
        imposter_scores = clf.decision_function(test_imposter)
        user_scores = list(user_scores)
        imposter_scores = list(imposter_scores)
        eers.append(evaluateEER(user_scores, imposter_scores))
    #print eers   
    return np.mean(eers), np.std(eers)

In [246]:
evaluate()

(0.81530951242962812, 0.088570174457551415)